In [43]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data")
docs = loader.load()
docs

[Document(page_content="Course Title Course Link Description Course length Learner Level\nGit & GitHub - The Practical Guide https://www.udemy.com/course/git-github-practical-guide/?couponCode=LETSLEARNNOWP Learn Git & GitHub and master working with commits, branches, the stash, cherry picking, rebasing, pull requests & more 10.5 hours All Level\nGit & GitHub for Beginners & Integration With Popular IDEs https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP Practical Approach to Version Control Git and GitHub. Learn How Git Works With IntelliJ, Pycharm, Visual Studio, Eclipse 4 hours All Level\nGIT: Advanced commands https://www.udemy.com/course/git-advanced-commands/?couponCode=LETSLEARNNOWPP Practical GIT training: cherry pick, rebase, revert, stash, clean, and more. Take your GIT skills to the next level 4 hours All Level\nGit & GitHub - The Complete Git & GitHub Course https://www.udemy.com/course/git-github-the-complete-git-

In [44]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 50)
documents = text_splitter.split_documents(docs)

In [45]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [46]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [47]:
db

In [48]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser



In [49]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [50]:
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [51]:
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")

In [52]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}. also provide Course link, duration and level of the course.

Prepare 3 sets of courses based on the total duration of the courses and each set should have 
at least 2 course.

The describution for the sets are following:
    - Set 1 Which we called "1 day plan", total duration of all the courses in set 1 should be less then 5 hours
    - Set 2 Which we called "3 day plan", total duration of all the courses in set 2 should be grater then 12 hours but less then 15 hours
    - Set 3 Which we called "5 day plan", total duration of all the courses in set 3 should be grater then 22 hours but less then 25 hours

Do not suggest courses which you have already suggested.

You must provide the following things in the answer:
    - Course Name
    - Course Link
    - Course Duration
    - Course Level



Be as concise as possible when giving answers. Do not anounce that you will answering. 
Do not say "Let's get started", or "Based on the skills you provided, I would recommend the 
following courses to help you enhance your knowledge and skills : " or "Here are some courses"
and do not introduce your answer. Just write the your answer.

<context>
{context}
<context>

skills : {skills}

experience : {experience}

"""
)

In [53]:
retriever = db.as_retriever()

In [54]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

In [55]:
chain = setup | prompt | llm | parser

In [56]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})

In [57]:
print(response)

Here are the recommended courses for each set:

**Set 1: 1-day plan**

* **Python for Everybody (3 hours)**: https://www.coursera.org/specializations/python-for-everybody
	+ Level: Beginner
	+ Duration: 3 hours
	+ Description: Learn the basics of Python programming
* **Introduction to Artificial Intelligence (2 hours)**: https://www.coursera.org/learn/introduction-artificial-intelligence
	+ Level: Beginner
	+ Duration: 2 hours
	+ Description: Learn the basics of Artificial Intelligence

**Set 2: 3-day plan**

* **Python Machine Learning (6 hours)**: https://www.coursera.org/specializations/python-machine-learning
	+ Level: Intermediate
	+ Duration: 6 hours
	+ Description: Learn the basics of Machine Learning with Python
* **Deep Learning with Python (6 hours)**: https://www.coursera.org/specializations/deep-learning-python
	+ Level: Intermediate
	+ Duration: 6 hours
	+ Description: Learn the basics of Deep Learning with Python

**Set 3: 5-day plan**

* **Python Data Structures (8 hours

In [58]:
response_2 = chain.invoke({"skills": "Java, AWS and Docker",
              "experience": 1})

print(response_2)

Here are three sets of courses based on the given context:

**Set 1: 1 day plan (total duration: 4.5 hours)**

* Course 1: "The Docker for DevOps Course from Development to Production" (8 hours, All Levels, https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/)
* Course 2: "DevOps Tools and AWS for Java Microservice Developers" (20.5 hours, All Levels, https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/)

**Set 2: 3 day plan (total duration: 14.5 hours)**

* Course 1: "The Docker for DevOps Course from Development to Production" (8 hours, All Levels, https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/)
* Course 2: "DevOps Tools and AWS for Java Microservice Developers" (20.5 hours, All Levels, https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/)

**Set 3: 5 day plan (total duration: 24 hours)**

* Course 1: "The Docker for DevOps Course from Developm

In [59]:
response_3 = chain.invoke({"skills": "DevOps and AWS",
              "experience": 2})

print(response_3)

Here are the suggested courses for each set:

**Set 1: 1 Day Plan**

1. The Docker for DevOps Course from Development to Production (8 hours, All Levels) - https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
2. DevOps Tools and AWS for Java Microservice Developers (20.5 hours, All Levels) - https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/

**Set 2: 3 Day Plan**

1. The Complete Cloud Computing Software Architecture Patterns (4.5 hours, All Levels) - https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
2. Design and Develop a Killer Website with HTML5 and CSS3 (37.5 hours, All Levels) - https://www.udemy.com/course/design-and-develop-a-killer-website-with-html5-and-css3/

**Set 3: 5 Day Plan**

1. DevOps: Jenkins CI/CD for Kubernetes Deployments (20.5 hours, All Levels) - https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
2. HTML and CSS for Begi

In [60]:
response_4 = chain.invoke({"skills": "DevOps",
              "experience": 2})

print(response_4)

Set 1: "1 day plan" (Total duration: 3.5 hours)

* Course Name: The Docker for DevOps Course from Development to Production
	+ Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	+ Course Duration: 8 hours
	+ Course Level: All Levels
* Course Name: DevOps Tools and AWS for Java Microservice Developers
	+ Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	+ Course Duration: 20.5 hours
	+ Course Level: All Levels

Set 2: "3 day plan" (Total duration: 14 hours)

* Course Name: Master Docker, Kubernetes, Maven, Jenkins, GIT, AWS EC2, Elastic Beanstalk, ELB, Auto Scaling and more in easy steps
	+ Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	+ Course Duration: 20.5 hours
	+ Course Level: All Levels
* Course Name: HTML and CSS for Beginners - Build a Website & Launch ONLINE
	+ Course Link: https://www.udemy.com/course/design-and-develop-a-kill

### EX 2:

In [99]:
prompt_2=ChatPromptTemplate.from_template(
"""
Suggest the best Talent Transformation courses based on provided {skills} and {experience}.

Prepare 3 sets of courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Classify courses by experience:
- Beginner: 0-1 years (suggest Beginner and All Level courses)
- Intermediate: 2-3 years (suggest Intermediate level courses)
- Advanced: 4-5 years (suggest Expert level courses)

create a "mandatory course" section and suggest at least 2 courses for that section from "mandatory course" document.

Each set should have at least 2 courses. Avoid repeating courses.

Requirements:
- Course Name
- Course Link
- Course Duration
- Course Level


<context>
{context}
<context>

Skills: {skills}

Experience: {experience}
"""
)


In [100]:
chain_2 = setup | prompt_2 | llm | parser

In [101]:
response = chain_2.invoke({"skills": "python and AI",
              "experience": 2})

print(response)

Based on the provided skills and experience, I've curated three sets of courses that cater to the learner's needs. Here are the suggested courses:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **The Docker for DevOps Course from Development to Production** (8 hours, All Levels)
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Description: Learn how to build and deploy web applications with Docker in an automated way.
2. **Cloud Computing Software Architecture Patterns** (4.5 hours, All Levels)
	* Course Link: https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* Course Description: Architect Large Scale Systems using Cloud Computing, Software Architecture Patterns & Modern System Design Principles

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Tools and AWS for Java Microservice Developers** (20.5 hours, All Levels)
	* Course Link: https://www

In [98]:
response_2 = chain_2.invoke({"skills": "Java, AWS and Docker",
              "experience": 1})

print(response_2)

Based on the provided skills and experience, I've curated three sets of courses for the given requirements. Since the person has 1 year of experience, I've categorized the courses as Beginner/All Level. Here are the suggested courses:

**Set 1 (1-day plan): Course length < 3 hours**

1. **The Docker for DevOps Course from Development to Production**
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Duration: 8 hours
	* Course Level: All Levels
2. **DevOps Tools and AWS for Java Microservice Developers**
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	* Course Duration: 20.5 hours
	* Course Level: All Levels

**Set 2 (3-day plan): Course length between 3 and 20 hours**

1. **DevOps Beginners to Advanced with Projects**
	* Course Link: https://www.udemy.com/course/devops-beginners-to-advanced-with-projects/
	* Course Duration: 4.5 hours
	* Course Level: All Levels
2. **T

In [94]:
response_3 = chain_2.invoke({"skills": "DevOps and AWS",
              "experience": 2})

print(response_3)

Based on the provided skills and experience, I've curated three sets of courses to help you transform your skills in DevOps and AWS. Here are the recommendations:

**Set 1 (1-day plan): Course length < 3 hours**

1. **DevOps Tools and AWS for Java Microservice Developers** (8 hours, All Levels) - https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
2. **The Docker for DevOps Course (From Development to Production)** (8 hours, All Levels) - https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/

**Set 2 (3-day plan): Course length between 3 and 20 hours**

1. **DevOps Beginners to Advanced with Projects** (12 hours, All Levels) - https://www.udemy.com/course/devops-beginners-to-advanced-with-projects/
2. **The Complete Cloud Computing Software Architecture Patterns** (4.5 hours, All Levels) - https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/

**Set 3 (5-day plan): Course length > 20

In [84]:
response_4 = chain_2.invoke({"skills": "DevOps",
              "experience": 5})

print(response_4)

Based on the provided skills and experience, I've curated three sets of courses for you. Each set has at least two courses, and I've included a mix of Git & GitHub, Linux, and SAP BTP courses as per the mandatory courses file.

**Set 1 (1-day plan): Course length < 3 hours**

1. **AWS Business Essentials - The Business Value of AWS** (2.5 hours, Beginner)
	* Course Link: https://www.udemy.com/course/aws-business-essentials/
	* Course Level: Beginner
2. **Introduction to Git and GitHub** (1.5 hours, Beginner)
	* Course Link: https://www.udemy.com/course/introduction-to-git-and-github-for-developers/ (Note: This course is not available; I'll suggest an alternative)
	* Alternative: **Git and GitHub for Beginners** (1.5 hours, Beginner)
		+ Course Link: https://www.udemy.com/course/git-and-github-for-beginners/

**Set 2 (3-day plan): Course length between 3 and 20 hours**

1. **DevOps Tools and AWS for Java Microservice Developers** (8 hours, All Levels)
	* Course Link: https://www.udemy.c

### Ex: 03

In [124]:
prompt_3=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}

Prepare 3 sets of courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Classify courses by experience:
- Beginner: 0-1 years (suggest Beginner and All Level courses)
- Intermediate: 2-3 years (suggest Intermediate level courses)
- Advanced: 4-5 years (suggest Expert level courses)

Ensure to include at least 2 courses of Git & GitHub, Linux and SAP BTP from the "mandatory courses" file.

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}
"""
)


In [125]:
chain_3 = setup | prompt_3 | llm | parser

In [126]:
response = chain_3.invoke({"skills": "python and AI",
              "experience": 2})

print(response)

Based on the provided context and skills, I've curated three sets of courses for you, categorized by total learning hours. I've also included at least two courses from the mandatory categories (Git & GitHub, Linux, and SAP BTP).

**Set 1 (1-day plan): Course duration < 3 hours**

1. **The Docker for DevOps Course** (2 hours, All Levels)
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Length: 8 total hours
	* Learner Level: All Levels
2. **The Complete Cloud Computing Software Architecture Patterns** (1.5 hours, All Levels)
	* Course Link: https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* Course Length: 4.5 total hours
	* Learner Level: All Levels

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Tools and AWS for Java Microservice Developers** (20.5 hours, All Levels)
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microser

In [127]:
response_2 = chain_3.invoke({"skills": "Java, AWS and Docker",
              "experience": 1})

print(response_2)

Based on the given context and skills, I've curated three sets of courses for you:

**Set 1 (1-day plan): Courses with duration < 3 hours**

1. **DevOps Tools and AWS for Java Microservice Developers**
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	* Course Length: 20.5 hours
	* Learner Level: All Levels
2. **The Docker for Devops Course: From Development to Production**
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Length: 8 hours
	* Learner Level: All Levels

**Set 2 (3-day plan): Courses with duration between 3 and 20 hours**

1. **The Complete Cloud Computing Software Architecture Patterns**
	* Course Link: https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* Course Length: 4.5 hours
	* Learner Level: All Levels
2. **DevOps Beginners to Advanced with Projects**
	* Course Link: https://www.udemy.com/course/devops-beginner

In [128]:
response_3 = chain_3.invoke({"skills": "DevOps and AWS",
              "experience": 2})

print(response_3)

Based on the provided context and skills, I've categorized the courses into three sets: 1-day plan, 3-day plan, and 5-day plan. I've also classified the courses by experience level. Here are the suggested courses:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Devops Tools and AWS for Java Microservice Developers**
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Length: 8 hours
	* Learner Level: All Levels
2. **The Complete Cloud Computing Software Architecture Patterns**
	* Course Link: https://www.udemy.com/course/the-complete-cloud-computing-software-architecture-patterns/
	* Course Length: 4.5 hours
	* Learner Level: All Levels

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **Master Docker, Kubernetes, Maven, Jenkins, GIT, AWS EC2, Elastic Beanstalk, ELB, Auto Scaling and more in easy steps**
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-devel

In [129]:
response_4 = chain_3.invoke({"skills": "DevOps",
              "experience": 5})

print(response_4)

As a helpful Talent Transformation assistant, I've analyzed the given context and created three sets of courses based on the total learning hours. I've also classified the courses by experience level and ensured the inclusion of mandatory courses.

**Set 1 (1-day plan): Course duration < 3 hours**

1. **AWS Business Essentials - The Business Value of AWS [2024]**
	* Course Link: https://www.udemy.com/course/aws-business-essentials/
	* Course Length: 2.5 hours
	* Learner Level: Beginner

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Tools and AWS for Java Microservice Developers**
	* Course Link: https://www.udemy.com/course/devops-tools-and-aws-for-java-microservice-developers/
	* Course Length: 20.5 hours
	* Learner Level: All Levels
2. **The Docker for DevOps Course from Development to Production**
	* Course Link: https://www.udemy.com/course/the-docker-for-devops-course-from-development-to-production/
	* Course Length: 8 hours
	* Learner Level: All Leve

## EX : 04 reducing chunk size

#### with chunk_size = 100, chunk_overlap= 20 ---->> highly halluciation

In [145]:
text_splitter_rc = RecursiveCharacterTextSplitter(chunk_size = 250, chunk_overlap = 50)
documents = text_splitter_rc.split_documents(docs)


db = FAISS.from_documents(documents, OllamaEmbeddings())


llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")



prompt_4=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}.


Prepare 3 sets of courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Classify courses by experience:
- Beginner: 0-1 years (suggest Beginner and All Level courses)
- Intermediate: 2-3 years (suggest Intermediate level courses)
- Advanced: 4-5 years (suggest Expert level courses)

Ensure to include at least 2 courses of Git & GitHub, Linux and SAP BTP from the "mandatory courses" file.

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

Suggest the courses only based on the context knowledge
Keep your course recomandation ground in the facts of the context.
If the context doesn’t contain the suitable courses according to the given {skills} then return "No course Found"

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}
"""
)



In [146]:
retriever = db.as_retriever()

parser = StrOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

chain = setup | prompt_4 | llm | parser

In [147]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})
print(response)


Based on the provided context and the skills and experience, I've curated three sets of courses for you. Here they are:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Python for Everybody** (1 hour) - All Levels
	* Course Link: https://www.datacamp.com/courses/python-for-everybody
	* Course Length: 1 hour
	* Learner Level: All Levels
2. **Introduction to Git and GitHub** (1 hour) - Beginner
	* Course Link: https://www.datacamp.com/courses/introduction-to-git-and-github
	* Course Length: 1 hour
	* Learner Level: Beginner

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **Python for Data Science Handbook** (12 hours) - All Levels
	* Course Link: https://www.datacamp.com/courses/python-for-data-science-handbook
	* Course Length: 12 hours
	* Learner Level: All Levels
2. **Data Science A-Z: Hands-On Exercises & ChatGPT Prize** (21 hours) - All Levels
	* Course Link: https://www.udemy.com/course/datascience/
	* Course Length: 21 hours
	* Learner Level: All Levels

In [148]:
response_2 = chain.invoke({"skills": "DevOps",
              "experience": 2})

print(response_2)


Based on the given context and skills, I've analyzed the courses and filtered them according to the provided criteria. Here are the suggested courses for each set:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Introduction to DevOps with Docker** (2 hours)
	* Course Link: https://www.udemy.com/course/introduction-to-devops-with-docker/
	* Course Length: 2 hours
	* Learner Level: All Levels
2. **Getting Started with Git and GitHub** (2 hours)
	* Course Link: https://www.udemy.com/course/getting-started-with-git-and-github/
	* Course Length: 2 hours
	* Learner Level: All Levels

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Engineer: DevOps AWS DevOps Docker DevOps Jenkins DevOpsGit, DevOps Vagrant DevOps Maven** (8.5 hours)
	* Course Link: https://www.udemy.com/course/devops-docker-complete-guide-hands-on-with-practical/
	* Course Length: 8.5 hours
	* Learner Level: All Levels
2. **Data Science A-Z: Hands-On Exercises & ChatGPT Prize [2024]** (21

In [149]:
response_3 = chain.invoke({"skills": "backend developer",
              "experience": 4})

print(response_3)

Based on the given context and the provided skills and experience, I will suggest three sets of courses, each with a different total learning time. I will also include at least two mandatory courses from the Git & GitHub, Linux, and SAP BTP categories.

Here are the suggested courses:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Git and GitHub for Developers**
	* Course Link: https://www.udemy.com/course/learn-git-and-github-from-scratch/
	* Course Length: 1.5 hours
	* Learner Level: Beginner
2. **Introduction to Linux Command Line**
	* Course Link: https://www.udemy.com/course/introduction-to-linux-command-line/
	* Course Length: 1.5 hours
	* Learner Level: Beginner

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **DevOps Engineer: DevOps, Docker, Jenkins, and Git**
	* Course Link: https://www.udemy.com/course/devops-docker-complete-guide-hands-on-with-practical/
	* Course Length: 8.5 hours
	* Learner Level: All Levels
2. **Backend Development with Pyth

In [150]:
response_4 = chain.invoke({"skills": "MLOps and AWS",
              "experience": 2})

print(response_4)

Based on the provided context and the given skills and experience, I've curated three sets of courses for you:

**Set 1 (1-day plan): Course duration < 3 hours**

1. **DevOps with Docker: A Complete Guide (Hands-on with Practical)** - https://www.udemy.com/course/devops-docker-complete-guide-hands-on-with-practical/
	* Course Length: 8.5 hours
	* Learner Level: All Levels

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **Data Science A-Z: Hands-On Exercises & ChatGPT Prize [2024]** - https://www.udemy.com/course/datascience/
	* Course Length: 21 hours
	* Learner Level: All Levels
2. **The Data Science Course: Complete Data Science Bootcamp** - https://www.udemy.com/course/the-data-science-course-complete-data-science-bootcamp/
	* Course Length: 32 hours
	* Learner Level: All Levels

**Set 3 (5-day plan): Course duration > 20 hours**

No courses found that meet this criteria.

As you have experience in MLOps and AWS with 2 years of experience, I've focused on courses

### Ex 5 : Increasing the chunk size

In [151]:
text_splitter_inc = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap = 150)
documents = text_splitter_inc.split_documents(docs)


db = FAISS.from_documents(documents, OllamaEmbeddings())


llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")



prompt_5=ChatPromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}.


Prepare 3 sets of courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Classify courses by experience:
- Beginner: 0-1 years (suggest Beginner and All Level courses)
- Intermediate: 2-3 years (suggest Intermediate level courses)
- Advanced: 4-5 years (suggest Expert level courses)

Ensure to include at least 2 courses of Git & GitHub, Linux and SAP BTP from the "mandatory courses" file.

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level

Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

Suggest the courses only based on the context knowledge
Keep your course recomandation ground in the facts of the context.
If the context doesn’t contain the suitable courses according to the given {skills} then return "No course Found"

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}
"""
)



In [152]:
retriever = db.as_retriever()

parser = StrOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

chain = setup | prompt_5 | llm | parser

In [153]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})
print(response)

Based on the given context and skills, I'd recommend the following courses for you:

**Set 1: 1-day plan (Courses with duration < 3 hours)**

1. **Microsoft Azure DevOps: Automate App Deployment From Scratch**
	* Course Link: https://www.udemy.com/course/master-azure-devops-from-scratch-automate-app-lifecycle/
	* Course Length: 4 hours
	* Learner Level: Intermediate
2. **Statistics for Data Science and Business Analysis**
	* Course Link: https://www.udemy.com/course/statistics-for-data-science-and-business-analysis/
	* Course Length: 5 hours
	* Learner Level: All Levels

**Set 2: 3-day plan (Courses with duration between 3 and 20 hours)**

1. **DevOps Projects: Master Real-Time DevOps & GitOps Projects**
	* Course Link: https://www.udemy.com/course/devopsprojects/
	* Course Length: 40 hours
	* Learner Level: Intermediate
2. **R Programming A-Z: R For Data Science With Real Exercises!**
	* Course Link: https://www.udemy.com/course/r-programming/
	* Course Length: 10.5 hours
	* Learner L

In [154]:
response_2 = chain.invoke({"skills": "DevOps",
              "experience": 2})

print(response_2)


Based on the provided context and skills, I've categorized the courses into three sets: Set 1 (1-day plan), Set 2 (3-day plan), and Set 3 (5-day plan) based on the course duration. I've also included at least two courses from the mandatory courses file, which are Git & GitHub, Linux, and SAP BTP.

**Set 1 (1-day plan): Course duration < 3 hours**

1. **Microsoft Azure DevOps: Automate App Deployment From Scratch**
	* Course Name: Microsoft Azure DevOps: Automate App Deployment From Scratch
	* Course Link: https://www.udemy.com/course/master-azure-devops-from-scratch-automate-app-lifecycle/
	* Course Length: 4 hours
	* Learner Level: Intermediate

**Set 2 (3-day plan): Course duration between 3 and 20 hours**

1. **Learn DevOps: Docker, Kubernetes, Terraform, and Azure DevOps**
	* Course Name: Learn DevOps: Docker, Kubernetes, Terraform, and Azure DevOps
	* Course Link: https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/
	* Course Length: 21.5 hours
	* Learner L

In [155]:
response_4 = chain.invoke({"skills": "MLOps and AWS",
              "experience": 2})

print(response_4)

Based on the given context, I've analyzed the courses and recommended the following sets of courses for you:

**Set 1 (1-day plan)**

1. **DevOps Tutorial: Complete Beginners Training - 5 in 1 Bundle**
	* Course Link: https://www.udemy.com/course/devops-docker-complete-guide-hands-on-with-practical/
	* Course Length: 8.5 hours
	* Learner Level: All Levels

**Set 2 (3-day plan)**

1. **Microsoft Azure DevOps :Automate App Deployment From Scratch**
	* Course Link: https://www.udemy.com/course/master-azure-devops-from-scratch-automate-app-lifecycle/
	* Course Length: 4 hours
	* Learner Level: Intermediate
2. **Learn DevOps: Docker, Kubernetes, Terraform and Azure DevOps**
	* Course Link: https://www.udemy.com/course/devops-with-docker-kubernetes-and-azure-devops/
	* Course Length: 21.5 hours
	* Learner Level: All Levels

**Set 3 (5-day plan)**

1. **Learn DevOps with Docker, Kubernetes, and Azure DevOps**
	* Course Link: https://www.udemy.com/course/devops-training/
	* Course Length: 71 h